# Cifar-100

In [1]:
from keras.layers import Dense, Dropout, Input, Conv2D, MaxPooling2D, Flatten, Activation
from keras.models import Sequential, Model
import keras
from keras.datasets import cifar100
import numpy as np
from keras import optimizers
from sklearn.model_selection import train_test_split
from keras.callbacks import CSVLogger
import tensorflow as tf
import innvestigate
import innvestigate.utils as iutils
import pandas as pd
import matplotlib.pyplot as plt
import os
import time
from collections import namedtuple
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [2]:
import setGPU

setGPU: Setting GPU to: 2


In [3]:
sess = tf.InteractiveSession()

In [4]:
random_state = 42

In [5]:
path_to_data = r'' + '../data/'

### 1 Loading cifar-100

In [6]:
def loading_cifar100(path_to_data, random_state):
    (x_train, y_train), (x_test, y_test) = cifar100.load_data()
    X_temp = np.asarray(np.concatenate(([x_train, x_test]))) 
    y_temp = np.asarray(np.concatenate(([y_train, y_test])))
    y_temp =  keras.utils.to_categorical(y_temp, 100)
    return X_temp, y_temp

In [7]:
X_temp, y_temp = loading_cifar100(path_to_data, random_state)

### Training the models

In [8]:
def create_cifar_alexnet():
    model = Sequential()

    # 1st Convolutional Layer
    model.add(Conv2D(filters=64, input_shape=(32,32,3), kernel_size=(11,11), strides=(4,4), padding='same'))
    model.add(Activation('relu'))
    # Max Pooling
    model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))

    # 2nd Convolutional Layer
    model.add(Conv2D(filters=192, kernel_size=(5,5), strides=(1,1), padding='same'))
    model.add(Activation('relu'))
    # Max Pooling
    model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))

    # 3rd Convolutional Layer
    model.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='same'))
    model.add(Activation('relu'))

    # 4th Convolutional Layer
    model.add(Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), padding='same'))
    model.add(Activation('relu'))

    # 5th Convolutional Layer
    model.add(Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), padding='same'))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))
    
    
    # Passing it to a Fully Connected layer
    model.add(Flatten())
    model.add(Dense(100))
    model.add(Activation('softmax'))
    return model

In [9]:
lr = 0.0001
epochs = 100
points_per_model = 30000
verbose = 0
experiment_name = "cifar-100"

In [10]:
def evaluate_model(model,X,y,explanation="gradient",batch_size=128):
    prediction = model.predict(X)
    loss = keras.losses.categorical_crossentropy(tf.convert_to_tensor(prediction),tf.convert_to_tensor(y))
    with sess.as_default():
        loss = loss.eval()
    try:
        model_wo_softmax = iutils.keras.graph.model_wo_softmax(model)
    except Exception:
        model_wo_softmax = model
    analyzer = innvestigate.create_analyzer(explanation, model_wo_softmax)
    analysis = np.zeros(X.shape)
    for batch in range(0,len(X),batch_size):
        analysis[batch:batch+batch_size] = analyzer.analyze(X[batch:batch+batch_size])
    analysis = np.reshape(analysis,[len(analysis),-1])
    prediction_var = np.var(prediction,axis=1)
    analysis_var = np.var(analysis,axis=1)
    analysis_1 = np.linalg.norm(analysis,axis=1,ord=1)
    analysis_2 = np.linalg.norm(analysis,axis=1,ord=2)
    return loss, prediction_var, analysis_var, analysis_1, analysis_2

In [11]:
for random_state in  range(1):
    print(random_state)
    print("######")
    path="ThresholdExperiments/{}/{}/".format(experiment_name,random_state)
    os.makedirs(path)
    np.random.seed(random_state)
    main_indicies = np.random.choice(len(X_temp),60000,replace=False)
    X,y = X_temp[main_indicies], y_temp[main_indicies]
    start = time.time()
    for model_number in range(np.floor(len(X)/(2*points_per_model)).astype(int)):
        print(model_number, end=" ")
        start_train = 2*model_number*points_per_model
        start_test = np.floor(start_train+(5/3)*points_per_model).astype(int)
        end_test = start_train+2*points_per_model
        x_train, x_test =  X[start_train:start_test], X[start_test:end_test]
        y_train, y_test =  y[start_train:start_test], y[start_test:end_test]
        optimizer = optimizers.Adam(lr=lr)
        model = create_cifar_alexnet()
        model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
        model.fit(x_train, y_train, epochs=epochs, 
      validation_data=(x_test, y_test), verbose=verbose)
        train_loss, train_prediction_var, train_analysis_var, train_analysis_1, train_analysis_2 = evaluate_model(model,x_train,y_train)
        df = pd.DataFrame()
        df['train_loss'] = train_loss
        df['train_prediction_var'] = train_prediction_var
        df['train_analysis_var'] = train_analysis_var
        df['train_analysis_1'] = train_analysis_1
        df['train_analysis_2'] = train_analysis_2
        df.to_csv("{}/{}_train.csv".format(path,model_number))
        test_loss, test_prediction_var, test_analysis_var, test_analysis_1, test_analysis_2 = evaluate_model(model,x_test,y_test)
        df = pd.DataFrame()
        df['test_loss'] = test_loss
        df['test_prediction_var'] = test_prediction_var
        df['test_analysis_var'] = test_analysis_var
        df['test_analysis_1'] = test_analysis_1
        df['test_analysis_2'] = test_analysis_2
        df.to_csv("{}/{}_test.csv".format(path,model_number))
        print(time.time() - start)

0
######


FileExistsError: [Errno 17] File exists: 'ThresholdExperiments/cifar-100/0/'